In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime,time

In [2]:
# Connect with mongodb

try:
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
    

    
db = conn['tweempact']

Connected successfully!!!


In [6]:
db.collection_names()

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  """Entry point for launching an IPython kernel.


['system.indexes', 'tweets_noRT', 'users_noRT', 'tweets', 'tweet_users']

In [3]:
def read_mongo(collections, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    import pandas as pd
    import pymongo
    """ Read from Mongo and Store into DataFrame """
    # Connect with mongodb

    try:
        with open("credentials.txt", 'r', encoding='utf-8') as f:
            [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
        print ("Connected successfully!!!")
    except pymongo.errors.ConnectionFailure as e:
        print ("Could not connect to MongoDB: %s" % e) 
     
    db = conn['tweempact']
    df1 = pd.DataFrame()
    for collection in collections:
        cursor = db[collection].find(query)
        df =  pd.DataFrame(list(cursor))
        if no_id:
            del df['_id']
        df1 = df1.append(df)
    df1 = df1.reset_index(drop = True)
    return df1

In [30]:
df = read_mongo(["tweets", "tweets_noRT"])

Connected successfully!!!


C:\Users\Albert Prat\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [6]:
len(df)

39283

In [7]:
 def author_dataset_conversion(df):   
    import time
    import numpy as np
    import pandas as pd

    cleaned = df[["id", "created_at","entities", "extended_entities","favorite_count","retweet_count","source","text", "truncated","user","quoted_status_id"]]
    ids = [user["id"] for user in cleaned["user"]]
    cleaned["id_num"] = ids

    #preparing output variables
    output = pd.DataFrame()
    users = []
    mean_RT_last10,mean_FC_last10 = [],[]
    sd_RT,sd_FC  = [],[]
    y1,y2=[],[]

    for _id in np.unique(ids):
        subset = cleaned[cleaned["id_num"]==_id].reset_index(drop=True)

        #filtering tweets from all tweets and RT
        l = list(subset["text"])
        filtr = ["RT" not in i for i in l]
        subset = subset[pd.Series(filtr)]
        if len(subset)>=11:
            reindexed = subset["created_at"].reset_index(drop=True)
            #convert tweet date into python date
            subset["created_at_pydate"] = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(reindexed[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(reindexed))]
            subset = subset.sort_values(by="created_at_pydate").reset_index(drop=True)

            last10_tweets = subset[-11:-1]
            last_tweet = subset[-1:]
            #RT parameters last 10 tweets
            mean_RT_last10.append(np.mean(last10_tweets["retweet_count"]))
            sd_RT.append(np.std(last10_tweets["retweet_count"]))
            #FC parameters last 10 tweets
            mean_FC_last10.append(np.mean(last10_tweets["favorite_count"]))
            sd_FC.append(np.std(last10_tweets["favorite_count"]))
            #take json user
            j_user = subset[-1:]["user"].reset_index(drop=True)[0]
            users.append(j_user)
            #parameters of last tweet
            y1.append(np.mean(last_tweet["favorite_count"]))
            y2.append(np.mean(last_tweet["retweet_count"]))
    output["j_user"]=users
    output["RT_l10"]=mean_RT_last10
    output["sd_RT"]=sd_RT
    output["FC_l10"]=mean_FC_last10
    output["sd_FC"]=sd_FC
    output["FC"]=y1
    output["RT"]=y2
    return output

In [8]:
output2 = author_dataset_conversion(df)

C:\Users\Albert Prat\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
output2.to_csv(r"C:/Users/Albert Prat/Documents/tweempact/datasets/author.csv")

In [13]:
df3 = pd.read_csv(r"C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [61]:
def trending_topic_df(df, path="c:/Users/Albert Prat/chromedriver.exe", just_dates=False):    
    from selenium import webdriver
    import pandas as pd
    import numpy as np
    import datetime
    
    if just_dates==False:
    # preparing elements to iterate
        textdf = df["Tweet"]
        cadf = df["created_at"]
        cadf = [date[:10] for date in cadf]
        cadf = np.unique(cadf)
        cadf = [date.split("-") for date in cadf]
        iterable_for_adding_dummy = [date[2]+date[1]+date[0] for date in cadf]
        # to be able to work with <, >
    if just_dates == True:
        iterable_for_adding_dummy = df
    # setting the minimum date to look for and preparing the filters   
    u = [datetime.datetime.strptime(date, "%d%m%Y") for date in iterable_for_adding_dummy]
    threshold = datetime.datetime.strptime("01032015", "%d%m%Y")
    filtr = [u[i]>=threshold for i in range(len(u))]
    filtr1 = [u[i]<threshold for i in range(len(u))]
    
    iterable = np.array(iterable_for_adding_dummy)[filtr]

    # which of the unique dates is before threshold?
    indices = [i for i, x in enumerate(filtr1) if x == True]
    
    # entering to browser and select the date box to be inputted
    browser = webdriver.Chrome(path)
    browser.get("https://trendogate.com/")
    date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
    
    # preparing output lists
    error = []
    grepl = []
    out = []
    for i in range(len(iterable)):
        browser.find_element_by_xpath("//select[@name='place']/option[text()='Spain']").click()
        date_dd = browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/div[2]/input")
        if just_dates == False:
            date = iterable[i][:10]
        if just_dates == True:
            date = iterable[i]
        date_dd.send_keys(date)
        browser.find_element_by_xpath("//*[@id='userheaderbox']/div/form[2]/button").click()

        if len(browser.find_elements_by_class_name("alert")) == 1:
            error.append(date)
            out.append(None)
            pass
        else:
            b = browser.find_element_by_class_name("col-lg-8")
            lgi = b.find_elements_by_class_name("list-group-item")
            TT = [el.text for el in lgi]
            TT = ",".join(TT)
            out.append(TT)
    for index in indices:
        out.insert(index, None)
    dates = [date[:2]+"/"+date[2:4]+"/"+date[4:] for date in iterable_for_adding_dummy]
    tocsv = pd.DataFrame([dates, out]).T
    tocsv.columns=["date", "trending_topics"]
    return tocsv
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = old["created_at"]
    ca_old = [date[:10] for date in ca_old]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = [date[:10] for date in ca_new]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    
    trend = trending_topic_df(list(new_dates), just_dates = True)
    output = pd.concat([old, trend], axis = 0).reset_index(drop = True)
    return output

In [62]:
Ttopics = trending_topic_df(textdf)

In [64]:
Ttopics.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [48]:
def text_dataset(df):
    import re
    import pandas as pd
    l = list(df["text"])
    filtr = ["RT" not in i for i in l]
    df = df[filtr].reset_index(drop=True)
    
    Tweet =[re.sub("[(\n)(\r)]"," ",tweet) for tweet in df["text"]]
    entities = [tweet for tweet in df["entities"]]
    FC = [tweet for tweet in df["favorite_count"]]
    RT = [tweet for tweet in df["retweet_count"]]
    id_ = [tweet for tweet in df["id"]]
    j_user = [tweet for tweet in df["user"]]
    followers = [tweet["followers_count"] for tweet in df["user"]]
    
    CA = df["created_at"]
    CA_l = [time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(CA[i],'%a %b %d %H:%M:%S +0000 %Y'))for i in range(0,len(CA))]
    
    output = pd.DataFrame([id_,Tweet,j_user,CA_l,entities,followers, FC,RT]).T
    output.reset_index(drop=True)
    output.columns = ["id", "Tweet", "j_user","created_at", "entities", "followers_user", "FC","RT"]
    
    #trending topic df
    #TT_new = widen_TT_csv(output)
    #TT_new.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    #TT_new = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    return output

In [49]:
textdf = text_dataset(df)

In [51]:
textdf.shape

(32831, 8)

In [28]:
textdf.shape

(32831, 8)

In [62]:
#pd.DataFrame.to_csv(output2, "C:/Users/Albert Prat/Documents/tweempact/datasets/tweet.csv")

In [19]:
def widen_TT_csv(df):
    old = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")
    ca_old = old["created_at"]
    ca_old = [date[:10] for date in ca_old]
    ca_old = np.unique(ca_old)
    ca_old = [date.split("-") for date in ca_old]
    ca_old = [date[2]+date[1]+date[0] for date in ca_old]
    
    ca_new = df["created_at"]
    ca_new = [date[:10] for date in ca_new]
    ca_new = np.unique(ca_new)
    ca_new = [date.split("-") for date in ca_new]
    ca_new = [date[2]+date[1]+date[0] for date in ca_new]
    
    new_dates = set(ca_new) - set(ca_old)
    print(new_dates)
    trend = trending_topic_df(list(new_dates), just_dates = True)
    output = pd.concat([old, trend], axis = 0).reset_index(drop = True)
    return output

In [40]:
trending_topics = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [3]:
output.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")

In [82]:
def append_dummy_TT(df):
    import re
    import unidecode
    import pandas as pd
    import ast
    
    textdf = df["Tweet"]
    trending_topics = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\trending_topics.csv")
    TT = trending_topics["trending_topics"]
    indices = TT[TT.isnull()].index
    # get TT as a list of lists
    out = []
    for i in range(len(TT)):
        if i in indices:
            out.append(None)
        else:
            t = TT[i].split(",")
            out.append(t)
    # date management
    ca = list(df["created_at"])
    ca = [date[:10] for date in ca]
    sp_d = [date.split("-") for date in ca]
    sp_d = [date[2]+date[1]+date[0] for date in sp_d]
    # just keeping unique dates
    ca = list(trending_topics["date"])
    unique_date = np.unique(ca)
    unique_date = [date.split("/") for date in unique_date]
    unique_date = [date[2]+date[1]+date[0] for date in unique_date]
    # append sp_d for the sake of indexing
    df["spanish_date"]=sp_d
    grepl = []
    for i in range(len(sp_d)):
        ls = []
        date = df[df.index == i]["spanish_date"][i]
        tweet = textdf[i].lower()
        tweet = unidecode.unidecode(tweet)
        index = np.where(trending_topics["date"]==(sp_d[i][:2]+"/"+sp_d[i][2:4]+"/"+sp_d[i][4:]))
        TT = out[index[0][0]]
        if TT == None:
            grepl.append(None)
        else:
            for word in TT:
                word = word.lower()
                word = unidecode.unidecode(word)
                word1 = re.sub(" ", "", word)
                ls.append(bool(re.search(r"\b"+str(word)+r"\b"+r"|\b"+str(word1)+r"\b", tweet)))
            grepl.append(np.sum(ls))
    df = df.drop("spanish_date", axis = 1)
    df["trending_topic"]=grepl
    df.reset_index(drop = True)
    return df
    df10 = df
    
    df1 = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)
    dropped = []
    for column in df.columns:
        dtype = type(df[column][0])
        if dtype == dict:
            df[column] = df[column].astype(str)
            dropped.append(column)
    df = pd.concat([df1,df], axis = 0).drop_duplicates().reset_index(drop=True)
    df.reset_index(drop = True)
    return df

In [83]:
out2 = append_dummy_TT(textdf)

In [86]:
np.where(out2["trending_topic"]==1)

(array([   66,   509,   556,   563,   639,   640,   826,  1129,  1145,
         1179,  1189,  1392,  1393,  1398,  1771,  1783,  1797,  1990,
         2207,  2209,  2261,  2737,  2927,  2955,  3319,  3685,  4293,
         4473,  4655,  4795,  4929,  5362,  6096,  6102,  6103,  6288,
         6706,  6709,  6712,  7375,  7438,  7771,  8400,  8529,  8557,
         8585,  8769,  9066,  9322,  9524,  9526,  9872,  9962,  9963,
         9984, 10050, 10113, 10340, 10341, 10630, 10766, 10767, 10768,
        10769, 10770, 10994, 11202, 11622, 12286, 12364, 12368, 12688,
        12692, 12694, 12792, 12796, 12943, 13001, 13010, 13128, 13276,
        13837, 13887, 14040, 14041, 14042, 14043, 14044, 14046, 14371,
        14476, 14542, 14679, 15020, 15179, 15240, 15577, 15579, 15581,
        15840, 16174, 16198, 16237, 16239, 17461, 17857, 17981, 17982,
        18105, 18166, 18290, 18307, 18312, 18314, 18315, 18666, 18687,
        18709, 19048, 19104, 19144, 19149, 19153, 19515, 19542, 19733,
      

In [93]:
out2[out2.index == 31111]["Tweet"][31111]

'Orange le arrebata República Móvil al grupo Masmóvil. ¿Qué razones hay para que el operador naranja haya cerrado es… https://t.co/SZadMFXfMa'

In [91]:
out2[out2.index == 31111]["created_at"][31111]

'2018-11-08 17:37:06'

In [95]:
out2.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

In [511]:
out = pd.read_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv", index_col=0)

In [514]:
len(out2)

42216

In [367]:
out.to_csv(r"C:\Users\Albert Prat\Documents\tweempact\datasets\tweet.csv")

In [72]:
out2.shape

(51791, 9)

In [75]:
out2.shape

(51791, 9)

In [76]:
textdf.shape

(32831, 9)

In [80]:
textdf["trending_topic"] = 

(32831, 9)